In [13]:
import tensorflow as tf
import PIL
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose, Conv2D, Dropout
from tensorflow.keras.optimizers import Adam

%matplotlib inline
%load_ext tensorboard
%load_ext pycodestyle_magic

1: E999 SyntaxError: invalid syntax
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [24]:
%flake8_on --max_line_length 115

BUFFLE_SIZE = 1000
BATCH_SIZE = 30
NOISE_DIM = 100
EPOCHS = 5
NUM_EXAMPLES = 16

seed = tf.random.normal([NUM_EXAMPLES, NOISE_DIM])

In [3]:
(images, labels), _ = tf.keras.datasets.mnist.load_data()
images = images.reshape([images.shape[0], 28, 28, 1])
images = tf.image.convert_image_dtype(images, tf.float32) * 2 - 1

train_ds = tf.data.Dataset.from_tensor_slices(images).shuffle(BUFFLE_SIZE).batch(BATCH_SIZE)

In [4]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(Dense(7*7*256, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Reshape((7, 7, 256)))

    model.add(Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model


def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(1))

    return model

generator = make_generator_model()
discriminator = make_discriminator_model()

In [ ]:
noise = tf.random.normal([1, NOISE_DIM])
generated_image = generator(noise, training=False)
decision = discriminator(generated_image)

plt.imshow(tf.squeeze(generated_image), cmap='viridis')
print(decision)

In [12]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)


def discriminator_loss(real_pred, fake_pred):
    real_loss = cross_entropy(tf.ones_like(real_pred), real_pred)
    fake_loss = cross_entropy(tf.zeros_like(fake_pred), fake_pred)

    return real_loss + fake_loss
    

def generator_loss(fake_pred):
    return cross_entropy(tf.ones_like(fake_pred), fake_pred)

In [27]:
checkpoint_dir = './GAN_checkpoints'
checkpoint_prex = checkpoint_dir + '/ckpt'
checkpoint = tf.train.Checkpoint(generator_optimizer=Adam(1e-4),
                                 discrimator_optimizer=Adam(1e-4),
                                 generator=generator,
                                 discriminator=discriminator)

In [19]:
checkpoint_prex

'./GAN_checkpoints/ckpt'

In [23]:
%%time
print(1)

1
CPU times: user 115 µs, sys: 73 µs, total: 188 µs
Wall time: 145 µs
